In [ ]:
from datafin.aws import SecretsClient                 #type: ignore
from datafin.aws import S3Client                      #type: ignore

from datafin.utils import GmailClient                 #type: ignore

In [6]:
import plotly.graph_objects as go

secrets = SecretsClient()

gmail = GmailClient(
    secrets.get_gmail_address(),
    secrets.get_gmail_app_password()
)

s3 = S3Client(
    bucket_name = secrets.get_bucket_name(),
    aws_access_key_id = secrets.aws_access_key,
    aws_secret_access_key = secrets.aws_secret_access_key,
)

df = s3.get_parquet(
    path = 'dev/polygon/equities/spy/store/market-open/year=2025',
    file_name = 'spy-open-2025-06-16'
)

fig = go.Figure(data=[
        go.Candlestick(
            x=df['datetime_ny'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close']
        )
    ]
)


fig.update_layout(xaxis_rangeslider_visible=False)
    
image_bytes = fig.to_image(
    format="png",
    scale=5,
    engine="kaleido"
)

s3.post_png(
    image_bytes=image_bytes,
    path="charts",
    file_name="candlestick_chart_v1"
)

image_from_s3 = s3.get_png(
    path = 'charts',
    file_name = 'candlestick_chart_v1'
)

gmail.send_email(
    to = secrets.get_gmail_send_to_address(),
    subject = 'todays opening spy minute',
    text = 'attached is an image of todays opening minute on spy',
    png_bytes = image_from_s3,
    png_filename = '06-16-opening-min'
)


True